# How to use the MongoDB checkpointer for persistence

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    Persistence
                </a>
            </li>       
            <li>
                <a href="https://www.mongodb.com/">
                    MongoDB
                </a>
            </li>        
        </ul>
    </p>
</div> 

When creating LangGraph agents, you can also set them up so that they persist their state. This allows you to do things like interact with an agent multiple times and have it remember previous interactions. 

This reference implementation shows how to use MongoDB as the backend for persisting checkpoint state using the `langgraph-checkpoint-mongodb` library.

For demonstration purposes we add persistence to a [prebuilt ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).

In general, you can add a checkpointer to any custom graph that you build like this:

```python
from langgraph.graph import StateGraph

builder = StateGraph(...)
# ... define the graph
checkpointer = # mongodb checkpointer (see examples below)
graph = builder.compile(checkpointer=checkpointer)
...
```

## Setup

To use the MongoDB checkpointer, you will need a MongoDB cluster. Follow [this guide](https://www.mongodb.com/docs/guides/atlas/cluster/) to create a cluster if you don't already have one.

Next, let's install the required packages and set our API keys

In [1]:
%%capture --no-stderr
%pip install -U pymongo langgraph langgraph-checkpoint-mongodb

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

## Define model and tools for the graph

In [ ]:
from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

In [4]:
@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## MongoDB checkpointer usage

### With a connection string

This creates a connection to MongoDB directly using the connection string of your cluster. This is ideal for use in scripts, one-off operations and short-lived applications.

In [33]:
from langgraph.checkpoint.mongodb import MongoDBSaver

In [ ]:
MONGODB_URI = "your_mongodb_connection_string"

In [37]:
with MongoDBSaver.from_conn_string(MONGODB_URI) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    response = graph.invoke(
        {"messages": [("human", "what's the weather in sf")]}, config
    )

In [38]:
response

{'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='62bfd903-af71-4099-958b-aaa70cd5e9cc'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UsoXhnO7TonK5WhYOUszVoUz', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-32b8432e-22e2-4527-a4f7-7f3dec33457b-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_UsoXhnO7TonK5WhYOUszVoUz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, '

### Using the MongoDB client

This creates a connection to MongoDB using the MongoDB client. This is ideal for long-running applications since it allows you to reuse the client instance for multiple database operations without needing to reinitialize the connection each time.

In [39]:
from pymongo import MongoClient

In [40]:
mongodb_client = MongoClient(MONGODB_URI)
checkpointer = MongoDBSaver(mongodb_client)
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
response = graph.invoke({"messages": [("user", "What's the weather in sf?")]}, config)

In [41]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='2c7a1ed9-943a-4890-b5f3-72416522f14e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D0bg1WF17ycrVvW83GnwY1lg', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ea6eeb84-0252-4508-8350-c7221f882755-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_D0bg1WF17ycrVvW83GnwY1lg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 

In [42]:
# Retrieve the latest checkpoint for the given thread ID
# To retrieve a specific checkpoint, pass the checkpoint_id in the config
checkpointer.get_tuple(config)

CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efb8c02-2f0e-6088-8003-b68cc03e80cb'}}, checkpoint={'v': 1, 'ts': '2024-12-12T19:34:43.121759+00:00', 'id': '1efb8c02-2f0e-6088-8003-b68cc03e80cb', 'channel_values': {'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='2c7a1ed9-943a-4890-b5f3-72416522f14e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D0bg1WF17ycrVvW83GnwY1lg', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c9

### Using an async connection

This creates a short-lived asynchronous connection to MongoDB. 

Async connections allow non-blocking database operations. This means other parts of your application can continue running while waiting for database operations to complete. It's particularly useful in high-concurrency scenarios or when dealing with I/O-bound operations.

In [43]:
from langgraph.checkpoint.mongodb.aio import AsyncMongoDBSaver

In [45]:
async with AsyncMongoDBSaver.from_conn_string(MONGODB_URI) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "3"}}
    response = await graph.ainvoke(
        {"messages": [("user", "What's the weather in sf?")]}, config
    )

In [46]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='ef818e0c-4181-4e9f-8974-89d9046394a2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ASCUDZbrA23uMaoTFUnFgF2i', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8584402c-ea76-4790-a6e6-51b88db3b750-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_ASCUDZbrA23uMaoTFUnFgF2i', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 

### Using the async MongoDB client

This routes connections to MongoDB through an asynchronous MongoDB client.

In [47]:
from pymongo import AsyncMongoClient

In [55]:
async_mongodb_client = AsyncMongoClient(MONGODB_URI)
checkpointer = AsyncMongoDBSaver(async_mongodb_client)
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "4"}}
response = await graph.ainvoke(
    {"messages": [("user", "What's the weather in sf?")]}, config
)

In [56]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='13316819-abd4-400b-8ab7-23275edb805c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PYgkOXiTaHmnwYMlyBueoMlF', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d381d744-5871-4d03-a969-d455eb120bd7-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_PYgkOXiTaHmnwYMlyBueoMlF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 

In [ ]:
# Retrieve the latest checkpoint for the given thread ID
# To retrieve a specific checkpoint, pass the checkpoint_id in the config
latest_checkpoint = await checkpointer.aget_tuple(config)
print(latest_checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efb8c1b-1ad9-637e-8003-80aae321523f'}}, checkpoint={'v': 1, 'ts': '2024-12-12T19:45:52.091605+00:00', 'id': '1efb8c1b-1ad9-637e-8003-80aae321523f', 'channel_values': {'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='13316819-abd4-400b-8ab7-23275edb805c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PYgkOXiTaHmnwYMlyBueoMlF', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e